In [ ]:
# ALL IMPORTS
import os
import pandas as pd
import json
import requests
import numpy as np 
pd.set_option('display.max_rows', 500)

## Read in Data

### Aviation data from: https://www.transtats.bts.gov/Fields.asp
- Select `Download` under `Data Tools`
- Filter to `Year` you would like
- Select these additiona columns:
  - "Passengers"   
  - "OrigonCityName"
  - "OriginState"
  - "OriginStateFips"
  - "DestCityName"
  - "DestState"
  - "DestStateFips"
  - "Month"
- Select `Download`
- Rename file and move to wrkdir

### Airport Directory Data from: https://www.faa.gov/airports/airport_safety/airportdata_5010/ 

- Used to Map airport to county name
- Many manual updates required, see update notes at end of notebook
- There is a cleaned version of this file in the repo, but to get your own:
  - scroll down to `Location(s) Selection Form`
  - Select `Submit`
  - Go to downloads
- For a new Download, you will have to update the airport codes as shown in the notes at the bottom  

### NOTES:

- AK: n Small aerodromes are not mapped to proper IATA code, n depends on min_pax filter
- FL: Fort Jefferson is federal land sans county
- PR: No Counties
    

## FILL THESE OUT

In [ ]:
#Enter the year of the data you downloaded (used for timestamp)
year = "2019"

# Filter by minimum number of pax per month
min_pax = 100

### AVIATION DATA

In [ ]:
# Aviation Data
aviation_fn = "UScarrier_2019_all_months.csv"
wrkdir = os.getcwd()

df_full = pd.read_csv(f"{wrkdir}/{aviation_fn}", sep=",", converters={'PASSENGERS': lambda x: int(float(x))},engine='python')

In [ ]:
# Delete phantom column
df_full = df_full[df_full.columns.drop(list(df_full.filter(regex='Unnamed')))]

# Delete rows with zero pax
df = df_full[df_full['PASSENGERS'] > min_pax] 

# sort by month
df = df.sort_values('MONTH').reset_index(drop=True)

#Drop Saipan and local VI flights
df =df[df["ORIGIN_STATE_ABR"] != "TT"]
df =df[df["ORIGIN_STATE_ABR"] != "VI"]

#Delete ISN WIlliston Airfield (closed October 10, 2019)
df =df[df["ORIGIN"] != "ISN"]
df =df[df["DEST"] != "ISN"]

In [ ]:
# For lambda function
def timestamp(x, year):
    DD = "01"    
    MM = str(x)
    ts = f'{year}-{MM}-{DD}'
    return ts

In [ ]:
# Add timestamp
# YYYY-MM-DD
df["TIMESTAMP"] = df.MONTH.apply(lambda x: timestamp(x, year))

# Split to just city name
df["ORIGIN_CITY"] = df.ORIGIN_CITY_NAME.apply(lambda x: x.split(",")[0])
df["DEST_CITY"] = df.DEST_CITY_NAME.apply(lambda x: x.split(",")[0])

### AIRPORT FACILITY DIRECTORY

In [ ]:
# Read in Airport Facilities Directory data to get county name
afd_fn = f"{wrkdir}/airportFD.txt"
df_afd = pd.read_csv(afd_fn, sep="\t")

In [ ]:
#Build county dictionary
df_afd_county = pd.DataFrame(df_afd, columns = ['LocationID', 'County'])
county_dict = df_afd_county.set_index('LocationID').to_dict()
county_d = county_dict["County"]

In [ ]:
# Tester for Airport to County
tester = ["CAK", "AUS", "SAN", "NUW", "DLF", "LKE", "RBH"]
for test in tester:
    print(f'{test} is in {county_d[test]} County')

### COUNTY FIPS

In [ ]:
fips_fn = f"{wrkdir}/county_to_fips.csv"
df_fips = pd.read_csv(fips_fn , sep=",", converters={"FIPS County Code": lambda x: str(x)},engine='python')

In [ ]:
#Build county dictionary
df_fips = pd.DataFrame(df_fips, columns = ['FIPS County Code', 'County Name'])
fips_dict = df_fips.set_index('County Name').to_dict()
fips_d = fips_dict["FIPS County Code"]

### UPDATE DF AND WRITE TO CSV

In [ ]:
# ADD Counties for origin and destination
df_county = df.copy()
df_county["ORIGIN_COUNTY"] = df.ORIGIN.apply(lambda x: county_d.get(x, np.NaN))
df_county["DEST_COUNTY"] = df.DEST.apply(lambda x: county_d.get(x, np.NaN))

In [ ]:
# ADD County FIPS for origin and destination
df_fin = df_county.copy()
df_fin["ORIGIN_COUNTY_FIPS"] = df_fin.ORIGIN_COUNTY.apply(lambda x: fips_d.get(x, np.NaN))
df_fin["DEST_COUNTY_FIPS"] = df_fin.DEST_COUNTY.apply(lambda x: fips_d.get(x, np.NaN))

# Replace #NAME? with NaN (for Puerto Rico)
df_fin = df_fin.replace('#NAME?',np.NaN)
df_fin = df_fin.replace('#NAME?',np.NaN)

In [ ]:
# Match capitalization format
def cap_it(x):
    temp = str(x).split()
    tt= ""
    for t in temp:
        tt = tt + " " + t.capitalize()
        
    return tt.lstrip()

df_fin["ORIGIN_COUNTY"]=df_fin["ORIGIN_COUNTY"].apply(lambda x: cap_it(x))
df_fin["DEST_COUNTY"]=df_fin["DEST_COUNTY"].apply(lambda x: cap_it(x))

In [ ]:
# Lowercase and reorder the columns
col_up = ["TIMESTAMP","ORIGIN", "ORIGIN_CITY", "ORIGIN_STATE_ABR", "ORIGIN_COUNTY", "ORIGIN_COUNTY_FIPS",
           "DEST",   "DEST_CITY",   "DEST_STATE_ABR",   "DEST_COUNTY",   "DEST_COUNTY_FIPS","PASSENGERS"]

col_low = [x.lower() for x in col_up]
df_fin.columns = [x.lower() for x in df_fin.columns]
df_fin =df_fin[col_low]

In [ ]:
# Write to CSV
df_fin.to_csv(fr'{wrkdir}/airport_pax_traffic_year={year}_min_pax={min_pax}.csv', index = False)

#### WORKS BUT NOT USED: APIs for FIPS Lookup

In [ ]:
def get_fips(icao):
    
    # REF: https://positionstack.com/quickstart, file: API_key.txt
    # position stack for lat/long of Airport ICAO
    base = 'http://api.positionstack.com/v1/forward'
    #key = 'API KEY HERE'
    icao_t = icao
    end = " Airport"
    query = icao_t + end
    country ="US"
    output = 'json'
    limit = 1
    search_latlong = f'{base}?access_key={key}&query={query}&country={country}&output={output}&limit={limit}'

    resp_latlong = requests.get(search_latlong)
    resp_json = json.loads(resp_latlong.text)

    lat = resp_json["data"][0]["latitude"]
    lon = resp_json["data"][0]["longitude"]
    
    # From FCC, get FIPS county data
    base_fcc = "https://geo.fcc.gov/api/census/area?"
    search_fcc = f'{base_fcc}lat={lat}&lon={lon}&format=json'
    
    response_fcc = requests.get(search_fcc)
    resp_fcc = json.loads(response_fcc.text)
    
    county_name = resp_fcc['results'][0]['county_name']
    county_fips = resp_fcc['results'][0]['county_fips']
    
    icao_data = [county_name, county_fips]
        
    return icao_data

In [ ]:
# Use Aviation data df to create list(df.series)
d=set(df_fin.DEST.to_list())
o=set(df_fin.ORIGIN.to_list())
icaos = list(d.union(o))

In [ ]:
all_icaos = {}
for icao in icaos[:10]:
    all_icaos[icao]= get_fips(icao)
all_icaos    

#### NOT NEEDED: Top 50 Filtering

In [ ]:
# Top 50 cities by population
top_50_fn = "city_top50.csv"
df_top_50 = pd.read_csv(f"{wrkdir}/{top_50_fn}", sep=",")

# Filter to Top 50 cities by population
top_50_list = df_top_50.city.tolist()

df_full["ORIGIN_CITY"] = df_full.ORIGIN_CITY_NAME.apply(lambda x: city_name(x))
df_full["DEST_CITY"] = df_full.DEST_CITY_NAME.apply(lambda x: city_name(x))


# Airport ID Lookup
airport_ID_fn= "L_AIRPORT_ID.csv"
airport_ID = pd.read_csv(f"{wrkdir}/{airport_ID_fn}", sep=",")

### Airport Facilities Dir NOTES:

#### County name to FIPS:
- Delete `.` after `ST`

#### Updates to IATA vs ICAO in airportFD.txt:
state, from, to

- AK, RBH copy of 5Z9
- AK KLW copy AKW
- CA, TRK, TKF
- CA, CLD, CRQ
- CA, IZA, SQA
- MT, FCA, GPI
- AZ, AZA, IWA
- AZ, SCF, SDL
- AZ, NYL, YUM
- AZ, 1Z1, DQS
- AZ, AZC, AZ7
- PR, VQS, JRV
- PA, UNV, SCE
- MA, added UBF copy of CQX
- MA, added QMN copy of 1B9
- MO, added BKG copy of BBG
- GA, added QMA, copy of RYY
- GA, added LIY copy of LHW
- NC, JQF, USA
- NC, AKH, NC1
- NC,  added NC2
- NV, BVU, BLD
- NV, HSH copy of HND
- NV, NV05,NV5
- MI, SAW, MQT
- WA, S60, KEH
- WA, added LKE same as KEH
- WA, ORS, ESD
- WA, FHR, FRD
- WA, W33, FBS
- TX DNE copy of DFW
- FL RQZ copy of HRT
- FL, X44, MPB
- FL RBN has no ICAO....Fort Jefferson Island off Key West
- FL, DTS, DSI
- NY, POU, DQK * added row
- NY, 0B8, FID *
- NY, VWK, 5B2 *
- ND ISN: closed October 10, 2019
- SC, HXD, HHH *
- SC, SC1 added Beaufort MCAS 
- UT, UXR copy of UT25
- NJ, added PCT copy 39N
- NJ, added NJ1 copy 19N
- NM, TSM copy SKX

In [ ]:
# Hand-jam mapping of ICAO to County code:

w= df_res[df_res["DEST_COUNTY"]=="None"]
w.shape

# Switch to Origin also
w["DEST_STATE_ABR"].unique()

w[w["ORIGIN_STATE_ABR"]=="FL"]["ORIGIN"].unique()

w[w["DEST_STATE_ABR"]=="TN"]

In [ ]:
# Airports without County FIPS

df_fin[df_fin["ORIGIN_COUNTY_FIPS"]=='None'].shape

d=df_fin[df_fin["ORIGIN_COUNTY"]=='None'][df_fin["ORIGIN_STATE_ABR"]=="AK"]
d=df_fin[df_fin["ORIGIN_COUNTY"]=='None'][df_fin["ORIGIN_STATE_ABR"]=="AK"]

s=d["ORIGIN"]

v=set(s)

len(v)